# Implementation of the IR-subtracted process $u \bar{u} \rightarrow \gamma \gamma \gamma$

In [612]:
# ENVIRONMENT SETUP

SYMBOLICA_COMMUNITY_PATH = None # If not installed for the current python intepreter, specified e.g. "/Users/vjhirsch/HEP_programs/symbolica-community/python"
ROOT_DIR = '/Users/vjhirsch/Documents/Work/Projects/alphaLoop/git_alphaLoopMisc/AmplitudeSubtractionBenchmark'
#ROOT_DIR = os.getcwd()
import os
os.environ['SYMBOLICA_LICENSE'] = 'a1d86de9#68b44941#7dca8216-8afe-57c8-975e-03eb5e68e4ee'

## Dependencies

In [613]:
import sys
from pprint import pprint, pformat
import os
import pydot
from itertools import product
import math
from math import sqrt, pow
from IPython.display import Markdown as md

if SYMBOLICA_COMMUNITY_PATH is not None:
    sys.path.insert(0, SYMBOLICA_COMMUNITY_PATH)
from symbolica_community import Expression, S,E, Replacement
from symbolica_community.tensors import TensorName as \
    N,LibraryTensor,TensorNetwork,Representation,TensorStructure,TensorIndices,Tensor,Slot,TensorLibrary, ExecutionMode
from symbolica_community.algebraic_simplification import *
import symbolica_community.tensors as tensors

# Work around expressions given as strings containing the wrapping quotes
def Es(expr: str) -> Expression:
    return E(expr.replace('"',''), "_gammaloop")

hep_lib = TensorLibrary.hep_lib()

In [614]:
# import symbolica_community
# print(symbolica_community.__file__)
# print()

## HELAS polarization vectors

In [615]:

class WaveFunction(list):
    """a objet for a WaveFunction"""
    
    spin_to_size={0:1,
                  1:3,
                  2:6,
                  3:6,
                  4:18,
                  5:18}
    
    def __init__(self, spin= None, size=None):
        """Init the list with zero value"""
        
        if spin:
            size = self.spin_to_size[spin]
        list.__init__(self, [0]*size)
        

def ixxxxx(p,fmass,nhel,nsf):
    """Defines an inflow fermion."""
    
    fi = WaveFunction(2)
    
    fi[0] = complex(-p[0]*nsf,-p[3]*nsf)
    fi[1] = complex(-p[1]*nsf,-p[2]*nsf) 
    nh = nhel*nsf 
    if (fmass != 0.):
        pp = min(p[0],sqrt(p[1]**2 + p[2]**2 + p[3]**2 ))
        if (pp == 0.): 
            sqm = [sqrt(abs(fmass))]
            sqm.append(sign(sqm[0],fmass)) 
            ip = (1+nh)//2 
            im = (1-nh)//2 

            fi[2] = ip*sqm[ip]
            fi[3] = im*nsf*sqm[ip]
            fi[4] = ip*nsf*sqm[im]
            fi[5] = im*sqm[im]

        else:
            sf = [(1+nsf+(1-nsf)*nh)*0.5,(1+nsf-(1-nsf)*nh)*0.5]
            omega = [sqrt(p[0]+pp),fmass/(sqrt(p[0]+pp))]
            ip = (1+nh)//2
            im = (1-nh)//2
            sfomeg = [sf[0]*omega[ip],sf[1]*omega[im]]
            pp3 = max(pp+p[3],0.)
            if (pp3 == 0.):
                chi1 = complex(-nh,0.) 
            else:
                chi1 = complex(nh*p[1]/sqrt(2.*pp*pp3),\
                p[2]/sqrt(2.*pp*pp3))
            chi = [complex(sqrt(pp3*0.5/pp)),chi1]

            fi[2] = sfomeg[0]*chi[im]
            fi[3] = sfomeg[0]*chi[ip]
            fi[4] = sfomeg[1]*chi[im]
            fi[5] = sfomeg[1]*chi[ip] 
    
    else: 
        sqp0p3 = sqrt(max(p[0]+p[3],0.))*nsf
        if (sqp0p3 == 0.):
            chi1 = complex(-nhel*sqrt(2.*p[0]),0.)
        else:
            chi1 = complex(nh*p[1]/sqp0p3,p[2]/sqp0p3)
        chi = [complex(sqp0p3,0.),chi1]
        if (nh == 1):
            fi[2] = complex(0.,0.)
            fi[3] = complex(0.,0.)
            fi[4] = chi[0]
            fi[5] = chi[1] 
        else:
            fi[2] = chi[1]
            fi[3] = chi[0]
            fi[4] = complex(0.,0.)
            fi[5] = complex(0.,0.) 
    
    return fi 

def oxxxxx(p,fmass,nhel,nsf):
    """ initialize an outgoing fermion"""
    
    fo = WaveFunction(2)
    fo[0] = complex(p[0]*nsf,p[3]*nsf)
    fo[1] = complex(p[1]*nsf,p[2]*nsf)
    nh = nhel*nsf
    if (fmass != 0.):
        pp = min(p[0],sqrt(p[1]**2 + p[2]**2 + p[3]**2 ))
        if (pp == 0.): 
            sqm = [sqrt(abs(fmass))]
            sqm.append( sign(sqm[0], fmass)) 
            ip = int(-((1-nh)//2) * nhel)
            im = int((1+nh)//2 * nhel)
            
            fo[2] = im*sqm[abs(im)]
            fo[3] = ip*nsf*sqm[abs(im)]
            fo[4] = im*nsf*sqm[abs(ip)]
            fo[5] = ip*sqm[abs(ip)]

        else:
            sf = [(1+nsf+(1-nsf)*nh)*0.5,(1+nsf-(1-nsf)*nh)*0.5]
            omega = [sqrt(p[0]+pp),fmass/(sqrt(p[0]+pp))]
            ip = (1+nh)//2
            im = (1-nh)//2
            sfomeg = [sf[0]*omega[ip],sf[1]*omega[im]]
            pp3 = max(pp+p[3],0.)
            if (pp3 == 0.):
                chi1 = complex(-nh,0.) 
            else:
                chi1 = complex(nh*p[1]/sqrt(2.*pp*pp3),\
                -p[2]/sqrt(2.*pp*pp3))
            chi = [complex(sqrt(pp3*0.5/pp)),chi1]

            fo[2] = sfomeg[1]*chi[im]
            fo[3] = sfomeg[1]*chi[ip]
            fo[4] = sfomeg[0]*chi[im]
            fo[5] = sfomeg[0]*chi[ip] 
            
    else: 
        sqp0p3 = sqrt(max(p[0]+p[3],0.))*nsf
        if (sqp0p3 == 0.):
            chi1 = complex(-nhel*sqrt(2.*p[0]),0.)
        else:
            chi1 = complex(nh*p[1]/sqp0p3,-p[2]/sqp0p3)
        chi = [complex(sqp0p3,0.),chi1]
        if (nh == 1):
            fo[2] = chi[0]
            fo[3] = chi[1]
            fo[4] = complex(0.,0.)
            fo[5] = complex(0.,0.) 
        else:
            fo[2] = complex(0.,0.)
            fo[3] = complex(0.,0.)
            fo[4] = chi[1]
            fo[5] = chi[0] 
    
    return fo

def vxxxxx(p,vmass,nhel,nsv):
    """ initialize a vector wavefunction. nhel=4 is for checking BRST"""
    
    vc = WaveFunction(3)
    
    sqh = sqrt(0.5)
    nsvahl = nsv*abs(nhel)
    pt2 = p[1]**2 + p[2]**2 
    pp = min(p[0],sqrt(pt2 + p[3]**2))
    pt = min(pp,sqrt(pt2))

    vc[0] = complex(p[0]*nsv,p[3]*nsv)
    vc[1] = complex(p[1]*nsv,p[2]*nsv)

    if (nhel == 4):
        if (vmass == 0.):
            vc[2] = 1.
            vc[3]=p[1]/p[0]
            vc[4]=p[2]/p[0]
            vc[5]=p[3]/p[0]
        else:
            vc[2] = p[0]/vmass
            vc[3] = p[1]/vmass
            vc[4] = p[2]/vmass
            vc[5] = p[3]/vmass
        
        return vc 

    if (vmass != 0.):
        hel0 = 1.-abs(nhel) 

        if (pp == 0.):
            vc[2] = complex(0.,0.)
            vc[3] = complex(-nhel*sqh,0.)
            vc[4] = complex(0.,nsvahl*sqh) 
            vc[5] = complex(hel0,0.)

        else:
            emp = p[0]/(vmass*pp)
            vc[2] = complex(hel0*pp/vmass,0.)
            vc[5] = complex(hel0*p[3]*emp+nhel*pt/pp*sqh)
            if (pt != 0.):
                pzpt = p[3]/(pp*pt)*sqh*nhel
                vc[3] = complex(hel0*p[1]*emp-p[1]*pzpt, \
                    -nsvahl*p[2]/pt*sqh)
                vc[4] = complex(hel0*p[2]*emp-p[2]*pzpt, \
                    nsvahl*p[1]/pt*sqh) 
            else:
                vc[3] = complex(-nhel*sqh,0.)
                vc[4] = complex(0.,nsvahl*sign(sqh,p[3]))
    else: 
        pp = p[0]
        pt = sqrt(p[1]**2 + p[2]**2)
        vc[2] = complex(0.,0.)
        vc[5] = complex(nhel*pt/pp*sqh)
        if (pt != 0.):
            pzpt = p[3]/(pp*pt)*sqh*nhel
            vc[3] = complex(-p[1]*pzpt,-nsv*p[2]/pt*sqh)
            vc[4] = complex(-p[2]*pzpt,nsv*p[1]/pt*sqh)
        else:
            vc[3] = complex(-nhel*sqh,0.)
            vc[4] = complex(0.,nsv*sign(sqh,p[3]))
    
    return vc

def sign(x,y):
    """Fortran's sign transfer function"""
    try:
        cmp = (y < 0.)
    except TypeError:
        # y should be complex
        if abs(y.imag) < 1e-6 * abs(y.real):
            y = y.real
        else:
            raise
    finally:
        if (y < 0.):
            return -abs(x) 
        else:
            return abs(x) 

def sxxxxx(p,nss):
    """initialize a scalar wavefunction"""
    
    sc = WaveFunction(1)
    
    sc[2] = complex(1.,0.)
    sc[0] = complex(p[0]*nss,p[3]*nss)
    sc[1] = complex(p[1]*nss,p[2]*nss)
    return sc


def txxxxx(p, tmass, nhel, nst):
    """ initialize a tensor wavefunction"""
    
    tc = WaveFunction(5)
    
    sqh = sqrt(0.5)
    sqs = sqrt(1/6)

    pt2 = p[1]**2 + p[2]**2
    pp = min(p[0],sqrt(pt2+p[3]**2))
    pt = min(pp,sqrt(pt2))

    ft = {}
    ft[(4,0)] = complex(p[0], p[3]) * nst
    ft[(5,0)] = complex(p[1], p[2]) * nst

    if ( nhel >= 0 ): 
        #construct eps+
        ep = [0] * 4
        
        if ( pp == 0 ):
            #ep[0] = 0
            ep[1] = -sqh
            ep[2] = complex(0, nst*sqh)
            #ep[3] = 0
        else:            
            #ep[0] = 0
            ep[3] = pt/pp*sqh
            if (pt != 0):
               pzpt = p[3]/(pp*pt)*sqh
               ep[1] = complex( -p[1]*pzpt , -nst*p[2]/pt*sqh )
               ep[2] = complex( -p[2]*pzpt ,  nst*p[1]/pt*sqh )
            else:
               ep[1] = -sqh 
               ep[2] = complex( 0 , nst*sign(sqh,p[3]) )
            
         
     
    if ( nhel <= 0 ): 
        #construct eps-
        em = [0] * 4
        if ( pp == 0 ):
            #em[0] = 0
            em[1] = sqh 
            em[2] = complex( 0 , nst*sqh )
            #em[3] = 0
        else:
            #em[0] = 0
            em[3] = -pt/pp*sqh
            if pt:
               pzpt = -p[3]/(pp*pt)*sqh
               em[1] = complex( -p[1]*pzpt , -nst*p[2]/pt*sqh )
               em[2] = complex( -p[2]*pzpt ,  nst*p[1]/pt*sqh )
            else:
               em[1] = sqh
               em[2] = complex( 0 , nst*sign(sqh,p[3]) )
            
    
    if ( abs(nhel) <= 1 ):  
        #construct eps0
        e0 = [0] * 4
        if ( pp == 0 ):
            #e0[0] = complex( rZero )
            #e0[1] = complex( rZero )
            #e0[2] = complex( rZero )
            e0[3] = 1
        else:
            emp = p[0]/(tmass*pp)
            e0[0] = pp/tmass 
            e0[3] = p[3]*emp
            if pt:
               e0[1] = p[1]*emp 
               e0[2] = p[2]*emp 
            #else:
            #   e0[1] = complex( rZero )
            #   e0[2] = complex( rZero )

    if nhel == 2:
        for j in range(4):
            for i in range(4):         
                ft[(i,j)] = ep[i]*ep[j]
    elif nhel == -2:
        for j in range(4):
            for i in range(4):         
                ft[(i,j)] = em[i]*em[j]
    elif tmass == 0:
        for j in range(4):
            for i in range(4):         
                ft[(i,j)] = 0
    elif nhel == 1:
        for j in range(4):
            for i in range(4): 
                ft[(i,j)] = sqh*( ep[i]*e0[j] + e0[i]*ep[j] )
    elif nhel == 0:
        for j in range(4):
            for i in range(4):       
                ft[(i,j)] = sqs*( ep[i]*em[j] + em[i]*ep[j] + 2 *e0[i]*e0[j] )
    elif nhel == -1:
        for j in range(4):
            for i in range(4): 
                ft[(i,j)] = sqh*( em[i]*e0[j] + e0[i]*em[j] )

    else:
        raise Exception('invalid helicity TXXXXXX') 



    tc[2] = ft[(0,0)]
    tc[3] = ft[(0,1)]
    tc[4] = ft[(0,2)]
    tc[5] = ft[(0,3)]
    tc[6] = ft[(1,0)]
    tc[7] = ft[(1,1)]
    tc[8] = ft[(1,2)]
    tc[9] = ft[(1,3)]
    tc[10] = ft[(2,0)]
    tc[11] = ft[(2,1)]
    tc[12] = ft[(2,2)]
    tc[13] = ft[(2,3)]
    tc[14] = ft[(3,0)]
    tc[15] = ft[(3,1)]
    tc[16] = ft[(3,2)]
    tc[17] = ft[(3,3)]
    tc[0] = ft[(4,0)]
    tc[1] = ft[(5,0)]

    return tc

def irxxxx(p, mass, nhel, nsr):
    """ initialize a incoming spin 3/2 wavefunction."""
    
    # This routines is a python translation of a routine written by
    # K.Mawatari in fortran dated from the 2008/02/26
    
    ri = WaveFunction(4)
    
    sqh = sqrt(0.5)
    sq2 = sqrt(2)
    sq3 = sqrt(3)
    
    pt2 = p[1]**2 + p[2]**2
    pp = min([p[0], sqrt(pt2+p[3]**2)])
    pt = min([pp,sqrt(pt2)])
    
    rc = {}
    rc[(4,0)] = -1*complex(p[0],p[3])*nsr
    rc[(5,0)] = -1*complex(p[1],p[2])*nsr
    

    nsv = -nsr # nsv=+1 for final, -1 for initial   
        
    # Construct eps+
    if nhel > 0:
        ep = [0] * 4
        if pp:
            #ep[0] = 0
            ep[3] = pt/pp*sqh
            if pt:
                pzpt = p[3]/(pp*pt)*sqh
                ep[1] = complex( -p[1]*pzpt , -nsv*p[2]/pt*sqh )
                ep[2] = complex( -p[2]*pzpt ,  nsv*p[1]/pt*sqh )
            else:
                ep[1] = -sqh 
                ep[2] = complex( 0 , nsv*sign(sqh,p[3]) )
        else:
            #ep[0] = 0d0
            ep[1] = -sqh
            ep[2] = complex(0, nsv * sqh)
            #ep[3] = 0d0

         
    if ( nhel < 0 ): 
        #construct eps-
        em = [0] * 4
        if ( pp == 0 ):
            #em[0] = 0
            em[1] = sqh 
            em[2] = complex( 0 , nsv*sqh )
            #em[3] = 0
        else:
            #em[0] = 0
            em[3] = -pt/pp*sqh
            if pt:
                pzpt = -p[3]/(pp*pt)*sqh
                em[1] = complex( -p[1]*pzpt , -nsv*p[2]/pt*sqh )
                em[2] = complex( -p[2]*pzpt ,  nsv*p[1]/pt*sqh )
            else:
                em[1] = sqh
                em[2] = complex( 0 , nsv*sign(sqh,p[3]) )            
                
    if ( abs(nhel) <= 1 ):  
        #construct eps0
        e0 = [0] * 4
        if ( pp == 0 ):
            #e0[0] = complex( rZero )
            #e0[1] = complex( rZero )
            #e0[2] = complex( rZero )
            e0[3] = 1
        else:
            emp = p[0]/(mass*pp)
            e0[0] = pp/mass 
            e0[3] = p[3]*emp
            if pt:
               e0[1] = p[1]*emp 
               e0[2] = p[2]*emp 
            #else:
            #   e0[1] = complex( rZero )
            #   e0[2] = complex( rZero )

    

    if ( nhel >= -1 ):
        # constract spinor+ 
        fip = [0] * 4
        sf, omega, sfomeg, chi = [0, 0], [0,0], [0,0], [0,0]
        nh = nsr
        if  mass:
            pp = min([p[0],sqrt(p[1]**2+p[2]**2+p[3]**2)])
            if pp == 0:
                sqm = sqrt(mass)
                ip = (1+nh)//2
                im = (1-nh)//2
                fip[0] = ip * sqm
                fip[1] = im * nsr * sqm
                fip[2] = ip * nsr * sqm
                fip[3] = im * sqm
            else:
                sf[0] = float(1+nsr+(1-nsr)*nh)*0.5
                sf[1] = float(1+nsr-(1-nsr)*nh)*0.5
                omega[0] = sqrt(p[0]+pp)
                omega[1] = mass/omega[0]
                ip = ((3+nh)//2) -1 # -1 since they are index 
                im = ((3-nh)//2) -1 # -1 since they are index
                sfomeg[0] = sf[0]*omega[ip]
                sfomeg[1] = sf[1]*omega[im]
                pp3 = max([pp+p[3],0])
                chi[0] = sqrt(pp3*0.5/pp)
                if  pp3 ==0:
                    chi[1] = -nh
                else:
                    chi[1] = complex( nh*p[1] , p[2] )/sqrt(2*pp*pp3)
            
                fip[0] = sfomeg[0]*chi[im]
                fip[1] = sfomeg[0]*chi[ip]
                fip[2] = sfomeg[1]*chi[im]
                fip[3] = sfomeg[1]*chi[ip]
        else:
            sqp0p3 = sqrt(max([p[0]+p[3],0])) * nsr
            chi[0] = sqp0p3
            if  sqp0p3 == 0:
                chi[1] = -nhel *  sqrt(2*p[0])
            else:
                chi[1] = complex( nh*p[1], p[2] )/sqp0p3
            if  nh == 1:
                #fip[0] = complex( rZero )
                #fip[1] = complex( rZero )
                fip[2] = chi[0]
                fip[3] = chi[1]
            else:
                fip[0] = chi[1]
                fip[1] = chi[0]
                #fip(3) = complex( rZero )
                #fip(4) = complex( rZero )
            
    if ( nhel <= 1 ):
        # constract spinor- 
        fim = [0] * 4
        sf, omega, sfomeg, chi = [0, 0], [0,0], [0,0], [0,0]
        nh = -nsr
        if  mass:
            pp = min([p[0],sqrt(p[1]**2+p[2]**2+p[3]**2)])
            if pp == 0:
                sqm = sqrt(mass)
                ip = (1+nh)/2
                im = (1-nh)/2
                fim[0] = ip * sqm
                fim[1] = im * nsr * sqm
                fim[2] = ip * nsr * sqm
                fim[3] = im * sqm
            else:
                sf[0] = float(1+nsr+(1-nsr)*nh)*0.5
                sf[1] = float(1+nsr-(1-nsr)*nh)*0.5
                omega[0] = sqrt(p[0]+pp)
                omega[1] = mass/omega[0]
                ip = (3+nh)//2 -1
                im = (3-nh)//2 -1
                sfomeg[0] = sf[0]*omega[ip]
                sfomeg[1] = sf[1]*omega[im]
                pp3 = max([pp+p[3],0])
                chi[0] = sqrt(pp3*0.5/pp)
                if  pp3 ==0:
                    chi[1] = -nh
                else:
                    chi[1] = complex( nh*p[1] , p[2] )/sqrt(2*pp*pp3)
            
                fim[0] = sfomeg[0]*chi[im]
                fim[1] = sfomeg[0]*chi[ip]
                fim[2] = sfomeg[1]*chi[im]
                fim[3] = sfomeg[1]*chi[ip]
        else:
            sqp0p3 = sqrt(max([p[0]+p[3],0])) * nsr
            chi[0] = sqp0p3
            if  sqp0p3 == 0:
                chi[1] = -nhel *  sqrt(2*p[0])
            else:
                chi[1] = complex( nh*p[1], p[2] )/sqp0p3
            if  nh == 1:
                #fip[0] = complex( rZero )
                #fip[1] = complex( rZero )
                fim[2] = chi[0]
                fim[3] = chi[1]
            else:
                fim[0] = chi[1]
                fim[1] = chi[0]
                #fip(3) = complex( rZero )
                #fip(4) = complex( rZero )        
      
    

    # recurent relation put her for optimization
    cond1  = (pt == 0 and p[3] >= 0)
    cond2  = (pt == 0 and p[3] < 0)
    
    # spin-3/2 fermion wavefunction
    if nhel == 3:
        for i,j in product(list(range(4)), list(range(4))):
            rc[(i, j)] = ep[i] *fip[j]
    
    elif nhel == 1:
        for i,j in product(list(range(4)), list(range(4))):
            if cond1:
                rc[(i,j)] = sq2/sq3*e0[i]*fip[j] +1/sq3*ep[i]*fim[j]
            elif cond2:
                rc[(i,j)] = sq2/sq3*e0[i]*fip[j] -1/sq3*ep[i]*fim[j]
            else:
                rc[(i,j)] = sq2/sq3*e0[i]*fip[j] + \
                                   1/sq3*ep[i]*fim[j] *complex(p[1],nsr*p[2])/pt  
    elif nhel == -1:
        for i,j in product(list(range(4)), list(range(4))):
            if cond1:
                rc[(i,j)] = 1/sq3*em[i]*fip[j] +sq2/sq3*e0[i]*fim[j]
            elif cond2:
                rc[(i,j)] = 1/sq3*em[i]*fip[j] -sq2/sq3*e0[i]*fim[j]
            else:
                rc[(i,j)] = 1/sq3*em[i]*fip[j] + \
                                sq2/sq3*e0[i]*fim[j] *complex(p[1],nsr*p[2])/pt  
    else:
        for i,j in product(list(range(4)), list(range(4))):
            if cond1:
                rc[(i, j)] = em[i] *fim[j]
            elif cond2:
                rc[(i, j)] = -em[i] *fim[j]
            else:
                rc[(i, j)] = em[i]*fim[j] *complex(p[1],nsr*p[2])/pt 
                
    ri[2] = rc[(0,0)]
    ri[3] = rc[(0,1)]
    ri[4] = rc[(0,2)]
    ri[5] = rc[(0,3)]
    ri[6] = rc[(1,0)]
    ri[7] = rc[(1,1)]
    ri[8] = rc[(1,2)]
    ri[9] = rc[(1,3)]
    ri[10] = rc[(2,0)]
    ri[11] = rc[(2,1)]
    ri[12] = rc[(2,2)]
    ri[13] = rc[(2,3)]
    ri[14] = rc[(3,0)]
    ri[15] = rc[(3,1)]
    ri[16] = rc[(3,2)]
    ri[17] = rc[(3,3)]
    ri[0] = rc[(4,0)]
    ri[1] = rc[(5,0)]              

    return ri

def orxxxx(p, mass, nhel, nsr):
    """ initialize a incoming spin 3/2 wavefunction."""
    
    # This routines is a python translation of a routine written by
    # K.Mawatari in fortran dated from the 2008/02/26

   
    ro = WaveFunction(spin=4)
    
    sqh = sqrt(0.5)
    sq2 = sqrt(2)
    sq3 = sqrt(3)
    
    pt2 = p[1]**2 + p[2]**2
    pp = min([p[0], sqrt(pt2+p[3]**2)])
    pt = min([pp,sqrt(pt2)])
    rc = {}
    rc[(4,0)] = complex(p[0],p[3])*nsr
    rc[(5,0)] = complex(p[1],p[2])*nsr
    

    nsv = nsr # nsv=+1 for final, -1 for initial   
        
    # Construct eps+
    if nhel > 0:
        ep = [0] * 4
        if pp:
            #ep[0] = 0
            ep[3] = pt/pp*sqh
            if pt:
                pzpt = p[3]/(pp*pt)*sqh
                ep[1] = complex( -p[1]*pzpt , -nsv*p[2]/pt*sqh )
                ep[2] = complex( -p[2]*pzpt ,  nsv*p[1]/pt*sqh )
            else:
                ep[1] = -sqh 
                ep[2] = complex( 0 , nsv*sign(sqh,p[3]) )
        else:
            #ep[0] = 0d0
            ep[1] = -sqh
            ep[2] = complex(0, nsv * sqh)
            #ep[3] = 0d0
         
    if ( nhel < 0 ): 
        #construct eps-
        em = [0] * 4
        if ( pp == 0 ):
            #em[0] = 0
            em[1] = sqh 
            em[2] = complex( 0 , nsv*sqh )
            #em[3] = 0
        else:
            #em[0] = 0
            em[3] = -pt/pp*sqh
            if pt:
                pzpt = -p[3]/(pp*pt)*sqh
                em[1] = complex( -p[1]*pzpt , -nsv*p[2]/pt*sqh )
                em[2] = complex( -p[2]*pzpt ,  nsv*p[1]/pt*sqh )
            else:
                em[1] = sqh
                em[2] = complex( 0 , nsv*sign(sqh,p[3]) )            
                
    if ( abs(nhel) <= 1 ):  
        #construct eps0
        e0 = [0] * 4
        if ( pp == 0 ):
            #e0[0] = complex( rZero )
            #e0[1] = complex( rZero )
            #e0[2] = complex( rZero )
            e0[3] = 1
        else:
            emp = p[0]/(mass*pp)
            e0[0] = pp/mass 
            e0[3] = p[3]*emp
            if pt:
               e0[1] = p[1]*emp 
               e0[2] = p[2]*emp 
            #else:
            #   e0[1] = complex( rZero )
            #   e0[2] = complex( rZero )

    if nhel >= -1:
        #constract spinor+ 
        nh = nsr
        sqm, fop, omega, sf, sfomeg = [0]*2,[0]*4,[0]*2,[0]*2,[0]*2
        chi = [0]*2
        if mass:
            pp = min([p[0],sqrt(p[1]**2+p[2]**2+p[3]**2)])
            if ( pp == 0):
                sqm[0] = sqrt(abs(mass)) # possibility of negative fermion masses
                sqm[1] = sign(sqm[0],mass) # possibility of negative fermion masses
                ip = -((1+nh)/2)
                im =  (1-nh)/2
                fop[0] = im     * sqm[im]
                fop[1] = ip*nsr * sqm[im]
                fop[2] = im*nsr * sqm[-ip]
                fop[3] = ip     * sqm[-ip]
            else:
                pp = min(p[0],sqrt(p[1]**2+p[2]**2+p[3]**2))
                sf[0] = (1+nsr+(1-nsr)*nh)*0.5
                sf[1] = (1+nsr-(1-nsr)*nh)*0.5
                omega[0] = sqrt(p[0]+pp)
                omega[1] = mass/omega[0]
                ip = (3+nh)//2  -1 # -1 since this is index
                im = (3-nh)//2  -1 # -1 since this is index 
                sfomeg[0] = sf[0]*omega[ip]
                sfomeg[1] = sf[1]*omega[im]
                pp3 = max([pp+p[3],0])
                chi[0] = sqrt(pp3*0.5/pp)
                if pp3 == 0:
                    chi[1] = -nh 
                else:
                    chi[1] = complex( nh*p[1] , -p[2] )/sqrt(2*pp*pp3)

            
                fop[0] = sfomeg[1]*chi[im]
                fop[1] = sfomeg[1]*chi[ip]
                fop[2] = sfomeg[0]*chi[im]
                fop[3] = sfomeg[0]*chi[ip]

        else:
            if(p[1] == 0 and p[2] == 0 and p[3] < 0):
                sqp0p3 = 0
            else:
                sqp0p3 = sqrt(max(p[0]+p[3], 0))*nsr
                
            chi[0] =  sqp0p3
            if ( sqp0p3 == 0 ):
                chi[1] = complex(-nhel )*sqrt(2*p[0])
            else:
                chi[1] = complex( nh*p[1], -p[2] )/sqp0p3
         
            if ( nh == 1 ):
                fop[0] = chi[0]
                fop[1] = chi[1]
                #fop[2] = 0
                #fop[3] = 0
            else:
                #fop[0] = 0
                #fop[1] = 0
                fop[2] = chi[1]
                fop[3] = chi[0]
         
    
    if ( nhel < 2 ):
        # constract spinor+ 
        sqm, fom, omega, sf, sfomeg = [0]*2,[0]*4,[0]*2,[0]*2,[0]*2
        chi = [0]*2

        
        nh = -nsr
        if mass:
            pp = min([p[0],sqrt(p[1]**2+p[2]**2+p[3]**2)])
            if ( pp == 0):
                sqm[0] = sqrt(abs(mass)) # possibility of negative fermion masses
                sqm[1] = sign(sqm[0],mass) # possibility of negative fermion masses
                ip = -((1+nh)/2)
                im =  (1-nh)/2
            
                fom[0] = im     * sqm[im]
                fom[1] = ip*nsr * sqm[im]
                fom[2] = im*nsr * sqm[-ip]
                fom[3] = ip     * sqm[-ip]
            
            else:
                pp = min([p[0],sqrt(p[1]**2+p[2]**2+p[3]**2)])
                sf[0] = (1+nsr+(1-nsr)*nh)*0.5
                sf[1] = (1+nsr-(1-nsr)*nh)*0.5
                omega[0] = sqrt(p[0]+pp)
                omega[1] = mass/omega[0]
                ip = (3+nh)//2 -1 #-1 since ip is an index
                im = (3-nh)//2 -1 
                sfomeg[0] = sf[0]*omega[ip]
                sfomeg[1] = sf[1]*omega[im]
                pp3 = max([pp+p[3], 0])
                chi[0] = sqrt(pp3*0.5/pp)
                if ( pp3 == 0):
                    chi[1] = -nh
                else:
                    chi[1] = complex( nh*p[1] , -p[2] )/sqrt(2*pp*pp3)
            
            
                fom[0] = sfomeg[1]*chi[im]
                fom[1] = sfomeg[1]*chi[ip]
                fom[2] = sfomeg[0]*chi[im]
                fom[3] = sfomeg[0]*chi[ip]
        else:
            if(p[1] == 0 == p[2] and p[3] < 0):
                sqp0p3 = 0
            else:
                sqp0p3 = sqrt(max([p[0]+p[3],0]))*nsr
            chi[0] = sqp0p3
            if ( sqp0p3 == 0):
                chi[1] = complex(-nhel )*sqrt(2*p[0])
            else:
                chi[1] = complex( nh*p[1], -p[2] )/sqp0p3
            if ( nh == 1 ):
                fom[0] = chi[0]
                fom[1] = chi[1]
                #fom[2] = 0
                #fom[3] = 0
            else:
                #fom[1] = 0
                #fom[2] = 0
                fom[2] = chi[1]
                fom[3] = chi[0]

    cond1 = ( pt==0 and p[3]>=0)
    cond2= (pt==0 and p[3]<0)

   
    # spin-3/2 fermion wavefunction
    if nhel == 3:
        for i,j in product(list(range(4)), list(range(4))):
            rc[(i, j)] = ep[i] *fop[j]  
    

    elif nhel == 1:
        for i,j in product(list(range(4)), list(range(4))):
            if cond1:
                rc[(i,j)] = sq2/sq3*e0[i]*fop[j] + 1/sq3*ep[i]*fom[j]
            elif cond2:
                rc[(i,j)] = sq2/sq3*e0[i]*fop[j] - 1/sq3*ep[i]*fom[j]
            else:
                rc[(i,j)] = sq2/sq3*e0[i]*fop[j] + 1/sq3*ep[i]*fom[j] * \
                                                      complex(p[1],-nsr*p[2])/pt  
                
    elif nhel == -1:
        for i,j in product(list(range(4)), list(range(4))):
            if cond1:
                rc[(i,j)] = 1/sq3*em[i]*fop[j]+sq2/sq3*e0[i]*fom[j]
            elif cond2:
                rc[(i,j)] =1/sq3*em[i]*fop[j]-sq2/sq3*e0[i]*fom[j]
            else:
                rc[(i,j)] =  1/sq3*em[i]*fop[j] + sq2/sq3*e0[i]*fom[j] *\
                                                      complex(p[1],-nsr*p[2])/pt              
    else:
        for i,j in product(list(range(4)), list(range(4))):
            if cond1:
                rc[(i,j)] = em[i] * fom[j]
            elif cond2:
                rc[(i,j)] = - em[i] * fom[j]
            else:
                rc[(i,j)] = em[i] * fom[j] * complex(p[1],-nsr*p[2])/pt 



    ro[2] = rc[(0,0)]
    ro[3] = rc[(0,1)]
    ro[4] = rc[(0,2)]
    ro[5] = rc[(0,3)]
    ro[6] = rc[(1,0)]
    ro[7] = rc[(1,1)]
    ro[8] = rc[(1,2)]
    ro[9] = rc[(1,3)]
    ro[10] = rc[(2,0)]
    ro[11] = rc[(2,1)]
    ro[12] = rc[(2,2)]
    ro[13] = rc[(2,3)]
    ro[14] = rc[(3,0)]
    ro[15] = rc[(3,1)]
    ro[16] = rc[(3,2)]
    ro[17] = rc[(3,3)]
    ro[0] = rc[(4,0)]
    ro[1] = rc[(5,0)]
    
    return ro

## Loading graphs

In [616]:
def get_numerator(g) -> Expression:
    num = E("1");
    for node in g.get_nodes():
        n_num = node.get("num")
        if n_num:
            num *= Es(n_num)
    for edge in g.get_edges():
        e_num = edge.get("num")
        if e_num:
            num *= Es(e_num)
    return num

def get_propagator_denominators(g) -> Expression:
    internal_nodes = [ n.get_name() for n in g.get_nodes() if not any(marker in n.get_name() for marker in ['graph', 'ext', 'edge', 'node']) ]
    den = E("1")
    for edge in g.get_edges():
        source = edge.get_source().split(':')[0]
        destination = edge.get_destination().split(':')[0]
        if source in internal_nodes and destination in internal_nodes:
            a_den = E(f'_gammaloop::Q({edge.get("id")},spenso::cind(0))^2')
            a_den -= E(f'_gammaloop::Q({edge.get("id")},spenso::cind(1))^2')
            a_den -= E(f'_gammaloop::Q({edge.get("id")},spenso::cind(2))^2')
            a_den -= E(f'_gammaloop::Q({edge.get("id")},spenso::cind(3))^2')
            den *= a_den
    return den

def get_projector(g) -> Expression:
    return Es(g.get_graph_defaults()[0]["projector"])

def get_emr_replacements(g) -> Expression:
    replacements = []
    for edge in g.get_edges():
        replacements.append((E(f"_gammaloop::Q({edge.get("id")},_gammaloop::a___)"),Es(edge.get("lmb_rep"))))
    return replacements

def tn_replace_multiple(tensor_network, replacements):
    res = tensor_network
    for lhs, rhs in replacements:
        res = res.replace(lhs, rhs)
    return res

In [617]:
tree_qqx_aaa_graphs = pydot.graph_from_dot_file(os.path.join(ROOT_DIR, "dot_graphs", "qqx_aaa_tree.dot"))

## Amplitude Evaluation

### Tools

In [618]:
def stepped_execution(tn, max_steps=None, t_delta=0.1):
    i_step = 0
    while True:
        i_step += 1
        print("Performing scalar step    #",i_step)
        import time
        if t_delta is not None:
            time.sleep(t_delta)
        print(tn)
        print(i_step)
        tn.execute(hep_lib, 1, ExecutionMode.Scalar)
        print("DONE")

        i_step += 1
        print("Performing reduction step #",i_step)
    
        print(tn)
        print(i_step)
        tn.execute(hep_lib, 1, ExecutionMode.Single)
        print("DONE")
    
        if max_steps and i_step > max_steps:
            break
        try:
            _ = tn.result_tensor(hep_lib)
        except:
            continue
        break

### Evaluation

In [619]:
def spdot(v1,v2):
    return v1[0]*v2[0]-v1[1]*v2[1]-v1[2]*v2[2]-v1[3]*v2[3]

def to_complex(cmplx):
    return E(str(complex(cmplx.real, cmplx.imag))).replace(E("python::j"),E("1𝑖"))

def function_map_for_evaluation(ks, hels, debug_pols=False):
    
    function_map = {
        E('_gammaloop::ee') : complex(0.30795376724436879,0.0),
    }
    
    for ext_id, k in enumerate(ks):
        for lor_i, ki in enumerate(k):
            function_map[E(f"_gammaloop::P({ext_id+1},spenso::cind({lor_i}))")] = complex(ki, 0.0)

    M2 = sqrt(abs(spdot(ks[2],ks[2])))
    if M2 < 1.0e-5:
        M2 = 0.0
    M3 = sqrt(abs(spdot(ks[3],ks[3])))
    if M3 < 1.0e-5:
        M3 = 0.0
    M4 = sqrt(abs(spdot(ks[4],ks[4])))
    if M4 < 1.0e-5:
        M4 = 0.0

    if debug_pols: print(ixxxxx(ks[0],0.0,hels[0],1)[2::])
    for lor_i, poli in enumerate(ixxxxx(ks[0],0.0,hels[0],1)[2::]):
        function_map[E(f"_gammaloop::u(1,spenso::cind({lor_i}))")]    = poli
    if debug_pols: print(oxxxxx(ks[1],0.0,hels[1],-1)[2::])
    for lor_i, poli in enumerate(oxxxxx(ks[1],0.0,hels[1],-1)[2::]):    
        function_map[E(f"_gammaloop::vbar(2,spenso::cind({lor_i}))")] = poli
        

    if debug_pols: print(vxxxxx(ks[2],sqrt(spdot(ks[2],ks[2])),hels[2],1)[2::])
    for lor_i, poli in enumerate(vxxxxx(ks[2],M2,hels[2],1)[2::]):
        function_map[E(f"_gammaloop::ϵbar(3,spenso::cind({lor_i}))")] = poli
    if debug_pols: print(vxxxxx(ks[3],sqrt(spdot(ks[3],ks[3])),hels[3],1)[2::])
    for lor_i, poli in enumerate(vxxxxx(ks[3],M3,hels[3],1)[2::]):
        function_map[E(f"_gammaloop::ϵbar(4,spenso::cind({lor_i}))")] = poli
    if debug_pols: print(vxxxxx(ks[4],sqrt(spdot(ks[4],ks[4])),hels[4],1)[2::])
    for lor_i, poli in enumerate(vxxxxx(ks[4],M4,hels[4],1)[2::]):
        function_map[E(f"_gammaloop::ϵbar(0,spenso::cind({lor_i}))")] = poli

    return function_map

In [620]:
kin_point = [
    [1.0, 0.0, 0.0, 1.0],
    [1.0, 0.0, 0.0, -1.0],
    [0.5, 0.3, 0.0, 0.0],
    [0.2, 0.0, 0.1, 0.1],
]
kin_point.append(
    [sum(k[i] for k in kin_point[:2])-sum(k[i] for k in kin_point[2:]) for i in range(4)]
)
helicities=[ 1, -1, -1, -1, 1 ]

kin_point2 = [
    [1.0, 0.0, 0.0, 1.0],
    [1.0, 0.0, 0.0, -1.0],
    [0.4, 0.1, 0.0, 0.0],
    [0.7, 0.0, 0.2, 0.6],
]
kin_point2.append(
    [sum(k[i] for k in kin_point2[:2])-sum(k[i] for k in kin_point2[2:]) for i in range(4)]
)
helicities2=[ 1, -1, -1, -1, 1 ]

kin_point_onshell = [
    [0.1000000000000000e+1,   0.0000000000000000e+0,   0.0000000000000000e+00,   0.1000000000000000e+01],
    [0.1000000000000000e+1,   0.0000000000000000e+0,   0.0000000000000000e+00,  -0.1000000000000000e+01],
    [0.9171575757708805e+0,   0.3389064406193597e+0,   0.7593073241563973e+00,  -0.3870049493005049e+00],
    [0.7281332414736353e+0,  -0.3665973858638370e-1,  -0.6954086026387343e+00,   0.2126992155174163e+00]
]
kin_point_onshell.append(
    [sum(k[i] for k in kin_point_onshell[:2])-sum(k[i] for k in kin_point_onshell[2:]) for i in range(4)]
)
helicities_onshell=[ 1, -1, -1, -1, 1 ]

kin_point2_onshell = [
    [0.1000000000000000e+1,   0.0000000000000000e+0,   0.0000000000000000e+0,   0.1000000000000000e+1,   0.0000000000000000e+0],
    [0.1000000000000000e+1,   0.0000000000000000e+0,   0.0000000000000000e+0,  -0.1000000000000000e+1,   0.0000000000000000e+0],
    [0.9903067841547669e+0,   0.3734458315384239e+0,   0.6393671561700484e+0,  -0.6576133949826120e+0,   0.2356080457693621e-7],
    [0.2053558277500182e+0,  -0.1412408546154445e+0,  -0.8691190591393230e-1,   0.1211129951276980e+0,   0.5101063824306858e-8]
]
kin_point2_onshell.append(
    [sum(k[i] for k in kin_point2_onshell[:2])-sum(k[i] for k in kin_point2_onshell[2:]) for i in range(4)]
)
helicities2_onshell=[ 1, -1, -1, -1, 1 ]

def display_sample_point(kins, hels, name=None):
    print(f"Sample point{'' if name is None else ' '+name}:")
    print('-'*150)
    for i, (k, h) in enumerate(zip(kins, hels)):
        print(f"#{i}    {k[0]:-25.16e} {k[1]:-25.16e} {k[2]:-25.16e} {k[3]:-25.16e} | m={sqrt(abs(spdot(k,k))):-25.16e} | hel={h:2d}")
    k_sum_in = [sum(k[i] for k in kins[:2]) for i in range(4)]
    k_sum_out = [sum(k[i] for k in kins[2:]) for i in range(4)]
    k_sum = [k_sum_in[i] - k_sum_out[i] for i in range(4)]
    print('-'*150)
    print(f"Sum:  {k_sum[0]:-25.16e} {k_sum[1]:-25.16e} {k_sum[2]:-25.16e} {k_sum[3]:-25.16e}")

# display_sample_point(kin_point, helicities, "PS point 1")
# print('')
# display_sample_point(kin_point2, helicities2, "PS point 2")
# print('')
display_sample_point(kin_point_onshell, helicities_onshell, "PS point onshell 1")
print('')
display_sample_point(kin_point2_onshell, helicities2_onshell, "PS point onshell 2")

# This corresponds then to the ordering (p1,p3,p4,p5,p2) when reading from the quark line from d to d~, which is diagram #2 in MadGraph ordering
# Targets as obtained from /Users/vjhirsch/MG5/MG5_aMC_v3_5_8/DDX_AAA_from_loops/SubProcesses/P0_ddx_aaa
MGTarget_kinpoin1 = complex(1.78588223425167137e-3,-1.16298831119980095e-3)
MGTarget_kinpoin2 = complex(7.67583136957218854e-4, 6.61541093750165753e-5)
MGTarget_kinpoin1_onshell = complex(-6.19639559102668088e-4,8.41535549668894736e-4)
MGTarget_kinpoin2_onshell = complex(1.47276041641056167e-4,-1.15031393691302202e-3)


Sample point PS point onshell 1:
------------------------------------------------------------------------------------------------------------------------------------------------------
#0       1.0000000000000000e+00    0.0000000000000000e+00    0.0000000000000000e+00    1.0000000000000000e+00 | m=   0.0000000000000000e+00 | hel= 1
#1       1.0000000000000000e+00    0.0000000000000000e+00    0.0000000000000000e+00   -1.0000000000000000e+00 | m=   0.0000000000000000e+00 | hel=-1
#2       9.1715757577088053e-01    3.3890644061935971e-01    7.5930732415639735e-01   -3.8700494930050489e-01 | m=   1.7473160325823090e-08 | hel=-1
#3       7.2813324147363534e-01   -3.6659738586383703e-02   -6.9540860263873427e-01    2.1269921551741630e-01 | m=   7.4505805969238281e-09 | hel=-1
#4       3.5470918275548424e-01   -3.0224670203297599e-01   -6.3898721517663071e-02    1.7430573378308858e-01 | m=   1.1632215228673813e-08 | hel= 1
-----------------------------------------------------------------------

Extract the full expression of the diagram:

In [621]:
tree_graph_0_expr = simplify_color(
    cook_indices(
        get_numerator(tree_qqx_aaa_graphs[0])
    )
    *
     E("(1/3)*spenso::g(spenso::cof(3,_gammaloop::hedge_1),spenso::dind(spenso::cof(3,_gammaloop::hedge_2)))")
)
#tree_graph_0_expr= simplify_gamma(tree_graph_0_expr)
tree_graph_0_expr *= cook_indices(get_projector(tree_qqx_aaa_graphs[0]))
tree_graph_0_expr /= get_propagator_denominators(tree_qqx_aaa_graphs[0])

# Fix current spenso issue
tree_graph_0_expr= tree_graph_0_expr.replace(Es("spenso::gamma(a_,b_,c_)"),Es("spenso::gamma(b_,a_,c_)")) 

In [622]:
# Experiment: compute a single wavefunction
# WF1expr= cook_indices(Es("""1
# *  (-1/3)*ee   
# *  u(1,spenso::bis(4,hedge(1)))
# *  spenso::gamma(spenso::bis(4,hedge_1),spenso::bis(4,hedge_5),spenso::mink(4,hedge_3))
# *  spenso::gamma(spenso::bis(4,hedge_5),spenso::bis(4,hedge_6),spenso::mink(4,edge_5_1))
# *  ϵbar(3,spenso::mink(4,hedge(3)))
# *  Q(5,spenso::mink(4,edge_5_1))
# *  (Q(5,spenso::cind(0))^2-Q(5,spenso::cind(1))^2-Q(5,spenso::cind(2))^2-Q(5,spenso::cind(3))^2)^-1
# """)).replace(Es("spenso::gamma(a_,b_,c_)"),Es("spenso::gamma(b_,a_,c_)"))
# WF1_tn = TensorNetwork(WF1expr, hep_lib)


# emr_replacements: Expression = get_emr_replacements(tree_qqx_aaa_graphs[0])
# WF1_tn = tn_replace_multiple(WF1_tn, emr_replacements)
# eval_function_map = function_map_for_evaluation(kin_point_onshell, helicities_onshell, debug_pols=True)
# WF1_tn = tn_replace_multiple(WF1_tn, [(lhs, to_complex(rhs)) for lhs, rhs in eval_function_map.items()] )
# WF1_tn.execute(hep_lib)
# wf_expr_tensor = WF1_tn.result_tensor()
# print(wf_expr_tensor)


# print(Es("Q(5,spenso::cind(3))").replace_multiple([Replacement(lhs, rhs) for lhs, rhs in emr_replacements]).evaluate_complex(eval_function_map,{}))



Prepare tensor networks for evaluation:

In [623]:
tn = TensorNetwork(tree_graph_0_expr, hep_lib)
tn.execute(hep_lib)
hel_amp_expr = tn.result_scalar()
emr_replacements = get_emr_replacements(tree_qqx_aaa_graphs[0])
hel_amp_expr_lmb = hel_amp_expr.replace_multiple([Replacement(lhs, rhs) for lhs, rhs in emr_replacements])
tn_emr = TensorNetwork(tree_graph_0_expr, hep_lib)
tn_lmb = tn_replace_multiple(tn_emr, emr_replacements)

Select test points:

In [624]:
# function_map_for_evaluation(kin_point, helicities)

test_point_A = {'kin': kin_point, 'hel' : helicities, 'MG_target': MGTarget_kinpoin1}
test_point_B = {'kin' : kin_point2, 'hel' : helicities2_onshell, 'MG_target': MGTarget_kinpoin2}


Now run the evaluations:

In [625]:
eval_function_map = function_map_for_evaluation(test_point_A['kin'], test_point_A['hel'])
#atom_eval = hel_amp_expr_lmb.replace_multiple([Replacement(lhs, to_complex(rhs)) for lhs, rhs in eval_function_map.items()])
atom_eval=to_complex(hel_amp_expr_lmb.evaluate_complex(eval_function_map,{}))
print("Atom eval: ", atom_eval)

Atom eval:  1.7858822342516714e-3-1.1629883111998005e-3𝑖


In [626]:
eval_function_map = function_map_for_evaluation(test_point_A['kin'], test_point_A['hel'])
tn_eval = tn_replace_multiple(tn_lmb, [(lhs, to_complex(rhs)) for lhs, rhs in eval_function_map.items()] )
tn_eval.execute(hep_lib)
eager_eval = tn_eval.result_scalar()
print("Eager eval: ",eager_eval)

Eager eval:  1.78588223425167e-3-1.16298831119980e-3𝑖


In [627]:
gL_PS1 = to_complex(hel_amp_expr_lmb.evaluate_complex(function_map_for_evaluation(test_point_A['kin'], test_point_A['hel']),{}))
print("gL_PSA: ", gL_PS1)
print("MG_PSA: ", to_complex(test_point_A['MG_target']))
print(gL_PS1 / to_complex(test_point_A['MG_target']))
gL_PS2 = to_complex(hel_amp_expr_lmb.evaluate_complex(function_map_for_evaluation(test_point_B['kin'], test_point_B['hel']),{}))
print("gL_PSB: ", gL_PS2)
print("MG_PSB: ", to_complex(test_point_B['MG_target']))
print(gL_PS2 / to_complex(test_point_B['MG_target']))

gL_PSA:  1.7858822342516714e-3-1.1629883111998005e-3𝑖
MG_PSA:  1.7858822342516714e-3-1.1629883111998010e-3𝑖
9.9999999999999989e-1+1.7e-16𝑖
gL_PSB:  7.675831369572191e-4+6.615410937501653e-5𝑖
MG_PSB:  7.675831369572189e-4+6.615410937501658e-5𝑖
1.000000000000000-8e-17𝑖
